# Experiment 112: Pseudo-Labeling for Distribution Adaptation

## Goal
Use pseudo-labeling to adapt the model to the test distribution.

## Key Insight
- ALL approaches fall on the SAME CV-LB line: LB = 4.09 × CV + 0.0546
- The intercept (0.0546) > target (0.0347) - target is UNREACHABLE with current approaches
- exp_111 (SimilarityAwareModel) LB=0.1063 vs expected 0.1074 - ON THE LINE
- We need something FUNDAMENTALLY DIFFERENT

## Pseudo-Labeling Strategy
1. Train initial model on training data
2. Predict on validation fold (unseen solvents)
3. Use confident predictions (low variance across seeds) as pseudo-labels
4. Augment training with pseudo-labels
5. Retrain model on augmented data

## Why This Might Work
- Test solvents are different from training solvents
- By using confident predictions on validation folds, we adapt to unseen solvents
- This could reduce extrapolation error (intercept)

In [1]:
import numpy as np
import pandas as pd
import torch
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add data path
sys.path.append('/home/data/')

from utils import (
    INPUT_LABELS_FULL_SOLVENT, INPUT_LABELS_SINGLE_SOLVENT, 
    INPUT_LABELS_NUMERIC, INPUT_LABELS_SINGLE_FEATURES, 
    INPUT_LABELS_FULL_FEATURES,
    generate_leave_one_out_splits, generate_leave_one_ramp_out_splits
)

# Override load functions to use local paths
DATA_PATH = '/home/data/'
TARGET_LABELS = ['Product 2', 'Product 3', 'SM']

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    assert name in ["spange_descriptors", "acs_pca_descriptors", "drfps_catechol", "fragprints", "smiles"]
    features = pd.read_csv(f'{DATA_PATH}{name}_lookup.csv', index_col=0)
    return features

print("Imports successful")

Imports successful


In [2]:
# Base classes and feature engineering
from abc import ABC, abstractmethod
from functools import reduce

torch.set_default_dtype(torch.double)

class SmilesFeaturizer(ABC):
    def __init__(self):
        raise NotImplementedError
    def featurize(X, Y):
        raise NotImplementedError

class BaseModel(ABC):
    def __init__(self):
        pass
    def train_model(self, X_train, y_train):
        raise NotImplementedError
    def predict(self):
        raise NotImplementedError

_SOLVENT_TABLE_CACHE = None

def feature_priority(name: str) -> int:
    if name.startswith("spange_"): return 5
    if name.startswith("acs_"): return 4
    if name.startswith("drfps_"): return 3
    if name.startswith("frag_"): return 2
    if name.startswith("smiles_"): return 1
    return 0

def filter_correlated_features(df, threshold=0.8):
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.shape[1] == 0:
        return df, []
    std = numeric_df.std(axis=0)
    constant_cols = std[std == 0].index.tolist()
    if constant_cols:
        numeric_df = numeric_df.drop(columns=constant_cols)
    corr = numeric_df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool)).fillna(0.0)
    cols = upper.columns.tolist()
    to_drop = set()
    high_corr_pairs = []
    for i, col_i in enumerate(cols):
        for j in range(i + 1, len(cols)):
            col_j = cols[j]
            cval = upper.iloc[i, j]
            if cval > threshold:
                high_corr_pairs.append((col_i, col_j, cval))
    for col_i, col_j, cval in high_corr_pairs:
        if col_i in to_drop or col_j in to_drop:
            continue
        p_i = feature_priority(col_i)
        p_j = feature_priority(col_j)
        if p_i > p_j:
            drop = col_j
        elif p_j > p_i:
            drop = col_i
        else:
            idx_i = df.columns.get_loc(col_i)
            idx_j = df.columns.get_loc(col_j)
            drop = col_i if idx_i > idx_j else col_j
        to_drop.add(drop)
    all_to_drop = list(set(constant_cols).union(to_drop))
    df_filtered = df.drop(columns=all_to_drop, errors="ignore")
    return df_filtered, all_to_drop

def add_numeric_features(X_numeric):
    X_num = X_numeric.copy()
    cols = set(X_num.columns)
    if {"Temperature", "Residence Time"} <= cols:
        X_num["Temperature"] = X_num["Temperature"] + 273.15
        T = X_num["Temperature"]
        rt = X_num["Residence Time"]
        X_num["T_x_RT"] = T * rt
        X_num["RT_log"] = np.log(rt + 1e-6)
        X_num["T_inv"] = 1 / T
        X_num["RT_scaled"] = rt / rt.mean()
    return X_num

print("Feature engineering functions defined")

Feature engineering functions defined


In [3]:
def build_solvent_feature_table(threshold=0.90):
    global _SOLVENT_TABLE_CACHE
    if _SOLVENT_TABLE_CACHE is not None:
        return _SOLVENT_TABLE_CACHE
    print(">>> Building solvent feature table...")
    sources = ["spange_descriptors", "acs_pca_descriptors", "drfps_catechol", "fragprints"]
    dfs = []
    for src in sources:
        df_src = load_features(src).copy()
        if "SOLVENT NAME" not in df_src.columns:
            df_src = df_src.reset_index().rename(columns={"index": "SOLVENT NAME"})
        if src in ["drfps_catechol", "fragprints"]:
            prefix = "drfps" if src == "drfps_catechol" else "frag"
            df_src = df_src.loc[:, (df_src != 0).any(axis=0)]
            df_src = df_src.loc[:, (df_src != 1).any(axis=0)]
            values = df_src.drop(columns={"SOLVENT NAME"})
            count = values.sum(axis=0).T
            drop_cols = count[count == 1].index
            df_src = df_src.drop(columns=drop_cols)
            cols_to_rename = [c for c in df_src.columns if c != "SOLVENT NAME"]
            df_src = df_src.rename(columns={c: f"{prefix}_{c}" for c in cols_to_rename})
        else:
            if src == "spange_descriptors":
                prefix = "spange"
            elif src == "acs_pca_descriptors":
                prefix = "acs"
            else:
                prefix = src
            cols_to_rename = [c for c in df_src.columns if c != "SOLVENT NAME"]
            df_src = df_src.rename(columns={c: f"{prefix}_{c}" for c in cols_to_rename})
        dfs.append(df_src)
    combined = reduce(lambda left, right: pd.merge(left, right, on="SOLVENT NAME", how="outer"), dfs)
    combined = combined.set_index("SOLVENT NAME")
    print(f"Combined feature table shape (before corr filter): {combined.shape}")
    combined, _ = filter_correlated_features(combined, threshold=threshold)
    print(f"Final solvent feature table shape: {combined.shape}")
    _SOLVENT_TABLE_CACHE = combined
    return combined

print("build_solvent_feature_table defined")

build_solvent_feature_table defined


In [4]:
# Featurizers with WEIGHTED AVERAGE for mixtures
class PrecomputedFeaturizer(SmilesFeaturizer):
    def __init__(self):
        self.featurizer = build_solvent_feature_table()
        dummy_num = pd.DataFrame([[0] * len(INPUT_LABELS_NUMERIC)], columns=INPUT_LABELS_NUMERIC)
        numeric_dim = add_numeric_features(dummy_num).shape[1]
        self.feats_dim = numeric_dim + self.featurizer.shape[1]

    def featurize(self, X):
        X_numeric = add_numeric_features(X[INPUT_LABELS_NUMERIC].copy())
        X_solvent = self.featurizer.loc[X["SOLVENT NAME"]]
        X_out = np.concatenate([X_numeric.values, X_solvent.values], axis=1)
        return torch.tensor(X_out, dtype=torch.double)

class PrecomputedFeaturizerMixed(SmilesFeaturizer):
    def __init__(self):
        self.featurizer = build_solvent_feature_table()
        dummy_num = pd.DataFrame([[0] * len(INPUT_LABELS_NUMERIC)], columns=INPUT_LABELS_NUMERIC)
        numeric_dim = add_numeric_features(dummy_num).shape[1]
        self.feats_dim = numeric_dim + self.featurizer.shape[1]

    def featurize(self, X):
        X_numeric = add_numeric_features(X[INPUT_LABELS_NUMERIC].copy())
        A = self.featurizer.loc[X["SOLVENT A NAME"]].values
        B = self.featurizer.loc[X["SOLVENT B NAME"]].values
        frac_B = X["SolventB%"].values.reshape(-1, 1)
        frac_A = 1 - frac_B
        mixed = A * frac_A + B * frac_B  # WEIGHTED AVERAGE
        X_out = np.concatenate([X_numeric.values, mixed], axis=1)
        return torch.tensor(X_out, dtype=torch.double)

print("Featurizers defined")

Featurizers defined


In [5]:
# CatBoost Model
from catboost import CatBoostRegressor

class CatBoostModel(BaseModel):
    def __init__(self, data="single", verbose=False, random_state=42):
        self.data_mode = data
        self.verbose = verbose
        self.random_state = random_state
        if data == "single":
            self.smiles_featurizer = PrecomputedFeaturizer()
            self.cat_params = dict(
                random_seed=random_state, loss_function="MultiRMSE",
                depth=3, learning_rate=0.07, n_estimators=1050,
                l2_leaf_reg=3.5, bootstrap_type="Bayesian",
                bagging_temperature=0.225, grow_policy="SymmetricTree",
                rsm=0.75, verbose=verbose,
            )
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed()
            self.cat_params = dict(
                random_seed=random_state, loss_function="MultiRMSE",
                depth=3, learning_rate=0.06, n_estimators=1100,
                l2_leaf_reg=2.5, bootstrap_type="Bayesian",
                bagging_temperature=0.2, grow_policy="SymmetricTree",
                rsm=0.7, verbose=verbose,
            )
        self.model = None

    def train_model(self, train_X, train_Y, device=None, verbose=False):
        X_tensor = self.smiles_featurizer.featurize(train_X)
        X_np = X_tensor.detach().cpu().numpy()
        Y_np = train_Y.values
        self.model = CatBoostRegressor(**self.cat_params)
        self.model.fit(X_np, Y_np)

    def predict(self, X):
        X_tensor = self.smiles_featurizer.featurize(X)
        X_np = X_tensor.detach().cpu().numpy()
        out = self.model.predict(X_np)
        out = np.clip(out, a_min=0.0, a_max=None)
        if out.ndim == 1:
            out = out.reshape(-1, 1)
        if out.shape[1] > 1:
            totals = out.sum(axis=1, keepdims=True)
            divisor = np.maximum(totals, 1.0)
            out = out / divisor
        return torch.tensor(out, dtype=torch.double)

print("CatBoostModel defined")

CatBoostModel defined


In [6]:
# XGBoost Model
from xgboost import XGBRegressor

class XGBModel(BaseModel):
    def __init__(self, data="single", random_state=42, verbose=False):
        self.data_mode = data
        self.verbose = verbose
        self.random_state = random_state
        if data == "single":
            self.smiles_featurizer = PrecomputedFeaturizer()
            self.xgb_params = dict(
                random_state=random_state, objective="reg:squarederror",
                tree_method="hist", subsample=0.5, reg_lambda=0.6,
                reg_alpha=0.0, n_estimators=1000, min_child_weight=1,
                max_depth=4, max_delta_step=1, learning_rate=0.02,
                grow_policy="depthwise", gamma=0.0, colsample_bytree=0.3,
                colsample_bylevel=0.6,
            )
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed()
            self.xgb_params = dict(
                random_state=random_state, objective="reg:squarederror",
                tree_method="approx", subsample=0.8, reg_lambda=0.5,
                reg_alpha=0.0, n_estimators=1200, min_child_weight=1,
                max_depth=5, max_delta_step=1, learning_rate=0.015,
                grow_policy="depthwise", gamma=0.0, colsample_bytree=0.4,
                colsample_bylevel=0.5,
            )
        self.models = None
        self.n_targets = None

    def train_model(self, train_X, train_Y, device=None, verbose=False):
        X_tensor = self.smiles_featurizer.featurize(train_X)
        X_np = X_tensor.detach().cpu().numpy()
        Y_np = train_Y.values
        self.n_targets = Y_np.shape[1]
        self.models = []
        for t in range(self.n_targets):
            m = XGBRegressor(**self.xgb_params)
            m.fit(X_np, Y_np[:, t])
            self.models.append(m)

    def predict(self, X):
        X_tensor = self.smiles_featurizer.featurize(X)
        X_np = X_tensor.detach().cpu().numpy()
        preds_list = [m.predict(X_np) for m in self.models]
        out = np.column_stack(preds_list)
        out = np.clip(out, a_min=0.0, a_max=None)
        if out.shape[1] > 1:
            totals = out.sum(axis=1, keepdims=True)
            divisor = np.maximum(totals, 1.0)
            out = out / divisor
        return torch.tensor(out, dtype=torch.double)

print("XGBModel defined")

XGBModel defined


In [7]:
# Ensemble Model
class EnsembleModel(BaseModel):
    def __init__(self, data="single", verbose=False, random_state=42):
        self.data_mode = data
        self.verbose = verbose
        self.random_state = random_state
        if data == "single":
            cat_weight = 7.0
            xgb_weight = 6.0
        else:
            cat_weight = 1.0
            xgb_weight = 2.0
        w_sum = cat_weight + xgb_weight
        self.cat_weight = cat_weight / w_sum
        self.xgb_weight = xgb_weight / w_sum
        self.cat_model = CatBoostModel(data=data, verbose=verbose, random_state=random_state)
        self.xgb_model = XGBModel(data=data, verbose=verbose, random_state=random_state)

    def train_model(self, train_X, train_Y, device=None, verbose=False):
        self.cat_model.train_model(train_X, train_Y, device, verbose)
        self.xgb_model.train_model(train_X, train_Y, device, verbose)

    def predict(self, X):
        cat_pred = self.cat_model.predict(X)
        xgb_pred = self.xgb_model.predict(X)
        out = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
        return out

print("EnsembleModel defined")

EnsembleModel defined


In [8]:
# PseudoLabelingModel - THE KEY INNOVATION
class PseudoLabelingModel(BaseModel):
    """
    Uses pseudo-labeling to adapt the model to unseen solvents.
    
    Strategy:
    1. Train initial ensemble on training data
    2. Predict on validation fold (unseen solvents)
    3. Use confident predictions (low variance across seeds) as pseudo-labels
    4. Augment training with pseudo-labels
    5. Retrain model on augmented data
    
    Why this might work:
    - Test solvents are different from training solvents
    - By using confident predictions on validation folds, we adapt to unseen solvents
    - This could reduce extrapolation error (intercept)
    """
    def __init__(self, data="single", n_seeds=5, confidence_percentile=50, verbose=False):
        self.data_mode = data
        self.n_seeds = n_seeds
        self.confidence_percentile = confidence_percentile
        self.verbose = verbose
        self.final_model = None

    def train_model(self, train_X, train_Y, device=None, verbose=False):
        """
        Train with pseudo-labeling:
        1. Train multiple models with different seeds
        2. Predict on training data to get variance
        3. Use low-variance predictions as pseudo-labels
        4. Retrain on augmented data
        """
        # Step 1: Train multiple models with different seeds
        models = []
        for seed in range(self.n_seeds):
            model = EnsembleModel(data=self.data_mode, random_state=seed*10+42)
            model.train_model(train_X, train_Y)
            models.append(model)
        
        # Step 2: Get predictions from all models on training data
        all_preds = []
        for model in models:
            preds = model.predict(train_X).numpy()
            all_preds.append(preds)
        
        all_preds = np.array(all_preds)  # Shape: (n_seeds, n_samples, n_targets)
        ensemble_mean = np.mean(all_preds, axis=0)  # Shape: (n_samples, n_targets)
        ensemble_var = np.var(all_preds, axis=0)  # Shape: (n_samples, n_targets)
        
        # Step 3: Identify confident predictions (low variance)
        mean_var = ensemble_var.mean(axis=1)  # Average variance across targets
        confidence_threshold = np.percentile(mean_var, self.confidence_percentile)
        confident_mask = mean_var < confidence_threshold
        
        if self.verbose:
            print(f"Confident samples: {confident_mask.sum()} / {len(confident_mask)}")
        
        # Step 4: Create pseudo-labels for confident samples
        # Use ensemble mean as pseudo-labels
        pseudo_labels = pd.DataFrame(
            ensemble_mean[confident_mask],
            columns=train_Y.columns,
            index=train_X.index[confident_mask]
        )
        
        # Step 5: Augment training data with pseudo-labels
        # Note: We're using the same samples but with smoothed labels
        # This is a form of label smoothing / self-training
        X_augmented = train_X.copy()
        Y_augmented = train_Y.copy()
        
        # Replace labels for confident samples with ensemble mean
        Y_augmented.loc[confident_mask] = pseudo_labels.values
        
        # Step 6: Train final model on augmented data
        self.final_model = EnsembleModel(data=self.data_mode, random_state=42)
        self.final_model.train_model(X_augmented, Y_augmented)

    def predict(self, X):
        return self.final_model.predict(X)

print("PseudoLabelingModel defined")

PseudoLabelingModel defined


In [9]:
# Quick CV evaluation
import tqdm

def evaluate_cv(model_class, **kwargs):
    """Evaluate using leave-one-out CV"""
    # Single solvent
    X, Y = load_data("single_solvent")
    split_generator = generate_leave_one_out_splits(X, Y)
    all_preds_single = []
    all_true_single = []
    
    for fold_idx, split in tqdm.tqdm(enumerate(split_generator), desc="single"):
        (train_X, train_Y), (test_X, test_Y) = split
        model = model_class(data='single', **kwargs)
        model.train_model(train_X, train_Y)
        predictions = model.predict(test_X)
        all_preds_single.append(predictions.numpy())
        all_true_single.append(test_Y.values)
    
    preds_single = np.vstack(all_preds_single)
    true_single = np.vstack(all_true_single)
    mse_single = np.mean((preds_single - true_single) ** 2)
    print(f"Single Solvent MSE: {mse_single:.6f}")
    
    # Full data
    X, Y = load_data("full")
    split_generator = generate_leave_one_ramp_out_splits(X, Y)
    all_preds_full = []
    all_true_full = []
    
    for fold_idx, split in tqdm.tqdm(enumerate(split_generator), desc="full"):
        (train_X, train_Y), (test_X, test_Y) = split
        model = model_class(data='full', **kwargs)
        model.train_model(train_X, train_Y)
        predictions = model.predict(test_X)
        all_preds_full.append(predictions.numpy())
        all_true_full.append(test_Y.values)
    
    preds_full = np.vstack(all_preds_full)
    true_full = np.vstack(all_true_full)
    mse_full = np.mean((preds_full - true_full) ** 2)
    print(f"Full Data MSE: {mse_full:.6f}")
    
    # Combined
    n_single = preds_single.shape[0] * preds_single.shape[1]
    n_full = preds_full.shape[0] * preds_full.shape[1]
    combined_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)
    print(f"\nCombined MSE (CV score): {combined_mse:.6f}")
    
    return combined_mse, mse_single, mse_full

print("Evaluation function defined")

Evaluation function defined


In [10]:
# First, verify baseline EnsembleModel CV
print("="*60)
print("Baseline EnsembleModel CV")
print("="*60)

baseline_cv, baseline_single, baseline_full = evaluate_cv(EnsembleModel)
print(f"\nBaseline CV: {baseline_cv:.6f}")
print(f"Expected LB from line: {4.09 * baseline_cv + 0.0546:.4f}")

Baseline EnsembleModel CV


single: 0it [00:00, ?it/s]

>>> Building solvent feature table...
Combined feature table shape (before corr filter): (26, 113)
Final solvent feature table shape: (26, 66)


single: 1it [00:01,  1.29s/it]

single: 2it [00:02,  1.08s/it]

...

full: 11it [00:32,  2.89s/it]

full: 12it [00:35,  2.91s/it]

full: 13it [00:38,  2.91s/it]

full: 13it [00:38,  2.94s/it]

Full Data MSE: 0.009895

Combined MSE (CV score): 0.009487

Baseline CV: 0.009487
Expected LB from line: 0.0934


In [ ]:
# Test PseudoLabelingModel
print("\n" + "="*60)
print("PseudoLabelingModel CV")
print("="*60)

pseudo_cv, pseudo_single, pseudo_full = evaluate_cv(PseudoLabelingModel, n_seeds=5, confidence_percentile=50)
print(f"\nPseudo-Labeling CV: {pseudo_cv:.6f}")
print(f"Expected LB from line: {4.09 * pseudo_cv + 0.0546:.4f}")
print(f"\nImprovement over baseline: {(baseline_cv - pseudo_cv) / baseline_cv * 100:.2f}%")

In [ ]:
# Save metrics
import json

metrics = {
    'cv_score': float(pseudo_cv),
    'mse_single': float(pseudo_single),
    'mse_full': float(pseudo_full),
    'baseline_cv': float(baseline_cv),
    'n_seeds': 5,
    'confidence_percentile': 50,
    'notes': 'PseudoLabelingModel - uses ensemble variance to identify confident predictions and smooth labels.'
}

with open('/home/code/experiments/112_pseudo_labeling/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Metrics saved")

## Submission Cells (CORRECT FORMAT)

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PseudoLabelingModel(data='single', n_seeds=5, confidence_percentile=50)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PseudoLabelingModel(data='full', n_seeds=5, confidence_percentile=50)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index(drop=True)
submission.index.name = "id"
submission.to_csv("/home/code/experiments/112_pseudo_labeling/submission.csv", index=True)

# Also copy to main submission folder
import shutil
shutil.copy("/home/code/experiments/112_pseudo_labeling/submission.csv", "/home/submission/submission.csv")

print(f"Submission shape: {submission.shape}")
print(f"Submission columns: {submission.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(submission.head())
print(f"\nLast 5 rows:")
print(submission.tail())

# Read back and verify format
sub_check = pd.read_csv("/home/submission/submission.csv")
print(f"\nRead back columns: {sub_check.columns.tolist()}")
expected_cols = ['id', 'task', 'fold', 'row', 'target_1', 'target_2', 'target_3']
assert list(sub_check.columns) == expected_cols, f"Wrong columns: {list(sub_check.columns)}"
print(f"\n✅ FORMAT VERIFIED: {expected_cols}")
print(f"\n✅ MODEL CLASS: PseudoLabelingModel (matches CV computation)")

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################